**Table of contents**<a id='toc0_'></a>    
- [DSPy](#toc1_)    
  - [Prediction](#toc1_1_)    
- [Signatures](#toc2_)    
- [Simple RAG](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[DSPy](#toc0_)


 Guide: YouTube - [Let the LLM Write the Prompts: An Intro to DSPy in Compound AI Pipelines](https://www.youtube.com/watch?v=I9ZtkgYZnOw)

In [13]:
#!pip install dspy

In [14]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

Getting keys from environment variables


In [15]:
lm = dspy.LM(
    'gpt-4.1',  # Changed from gpt-4o to gpt-3.5-turbo
    model_type='chat', 
    cache=False, 
    api_key=settings.OPENAI_API_KEY,
    temperature=0.8  
    
)

dspy.configure(lm=lm)

## <a id='toc1_1_'></a>[Prediction](#toc0_)
14m 52s in video

In [16]:
# Define a module predict and assign it a signature
predict = dspy.Predict('question -> answer')

In [17]:
# Call LLM for prediction
prediction = predict(question='Why is the sky blue?')

In [18]:
md(type(prediction))
md("**Answer**  ", "", prediction.answer)

<class 'dspy.primitives.prediction.Prediction'>

**Answer**  

The sky appears blue because of a phenomenon called Rayleigh scattering. As sunlight passes through Earth’s atmosphere, the shorter blue wavelengths of light are scattered in all directions by the gases and particles in the air more than other colors. This scattered blue light is what we see when we look up at the sky.

# <a id='toc2_'></a>[Signatures](#toc0_)

Location 16:43 in video

In [19]:
# Create classes

import dspy 
from pydantic import BaseModel
from typing import Literal

class Place(BaseModel):
    address: str = dspy.InputField()
    name: str = dspy.InputField()
    
class PlaceMatcher(dspy.Signature):
    """
    Verify that the text is based on the provided context.
    """
    # Inputs
    place_one: Place = dspy.InputField()
    place_two: Place = dspy.InputField()
    
    # Outputs
    is_match: bool = dspy.OutputField(desc="Do the two places refer to the same place?")
    match_confidence: Literal["low", "medium", "high"] = dspy.OutputField()
    rationale: str = dspy.OutputField(desc="Rationale for the match decision")

In [20]:
# Call LLM
predict = dspy.Predict(PlaceMatcher)
result = predict(place_one=Place(address="123 Main St, Springfield", name="Springfield Library"),
                 place_two=Place(address="123 Main St, Springfield", name="Springfield Public Library"))

In [21]:
md(result)

Prediction(  
    is_match=True,  
    match_confidence='high',  
    rationale='Both records have the exact same address ("123 Main St, Springfield"), and their names are highly similar, with only "Public" added in the second name. It is common for the same library to be referred to as "Springfield Library" and "Springfield Public Library." Given the matching address and near-identical names, it is highly likely they refer to the same place.'  
)

# <a id='toc3_'></a>[Simple RAG](#toc0_)

Uses Python Interpreter


In [22]:
# %pip install deno

In [23]:
# Setup

# Tools
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def customer_private_info(query):
    return "Super Secret Info"

my_tools = [evaluate_math]

# if user_in_role("super-user"):
#     my_tools.append(customer_private_info)

react = dspy.ReAct("question -> answer: float", tools=my_tools)

In [24]:
# Call LLM
result = react(question="What is the volume of Earth?")

In [25]:
print(type(result))
print(result)

<class 'dspy.primitives.prediction.Prediction'>
Prediction(
    trajectory={'thought_0': 'To find the volume of Earth, I will use the formula for the volume of a sphere: V = (4/3) * π * r^3. The average radius of Earth is approximately 6,371 km. I will substitute this value into the formula to compute the volume.', 'tool_name_0': 'evaluate_math', 'tool_args_0': {'expression': '(4/3) * 3.141592653589793 * (6371 ** 3)'}, 'observation_0': 'Execution error in evaluate_math: \nTraceback (most recent call last):\n  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/dspy/primitives/python_interpreter.py", line 124, in _ensure_deno_process\n    self.deno_process = subprocess.Popen(\n                        ^^^^^^^^^^^^^^^^^\n  File "/usr/local/python/3.12.1/lib/python3.12/subprocess.py", line 1026, in __init__\n    self._execute_child(args, executable, preexec_fn, close_fds,\n  File "/usr/local/python/3.12.1/lib/python3.12/subprocess.py", line 1950, in _execute_child\n    raise child_